In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
# !pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

In [3]:
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')

In [5]:
def data_processing(data):
    data = data.drop(['id'],axis=1)
    data['BMI'] = data['Weight'] / data['Height']**2
    numerical_columns = data.select_dtypes(include=['float64', 'int64'])
    object_columns = data.select_dtypes(include=['object'])
    
    binary_cols = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
    for col in binary_cols:
        data[col] = data[col].map({'yes': 1, 'no': 0})

    data['Gender'] = data['Gender'].map({'Male':1,'Female':0})
    data['CAEC'] = data['CAEC'].map({
        'no': 0,
        'Always': 3,
        'Frequently': 2,
        'Sometimes': 1
    })

    data['CALC'] = data['CALC'].map({
        'no': 0,
        'Frequently': 2,
        'Sometimes': 1
    })
    
    transport_mapping = {
        'Bike': 2,
        'Walking': 2,
        'Public_Transportation': 1,
        'Automobile': 0,
        'Motorbike': 0,
    }
    data['MTRANS'] = data['MTRANS'].map(transport_mapping)
    
    power_transformer = PowerTransformer(method='box-cox')
    data[numerical_columns.columns] = power_transformer.fit_transform(data[numerical_columns.columns]+1)
    scaler = StandardScaler()
    data[numerical_columns.columns] = scaler.fit_transform(data[numerical_columns.columns])
    return data

In [6]:
def train_processing(data):
    
    
    target_mapping = {
        'Insufficient_Weight': 0,
        'Normal_Weight': 1,
        'Overweight_Level_I': 2,
        'Overweight_Level_II': 3,
        'Obesity_Type_I': 4,
        'Obesity_Type_II': 5,
        'Obesity_Type_III': 6
    }

    data = data.map(target_mapping)
    return data

In [7]:
# X = train.drop(['id','NObeyesdad'],axis=1)
# X = np.array(df_train.drop(['NObeyesdad'], axis=1))
# y = np.array(df_train['NObeyesdad']) 
X = data_processing(train).drop(['NObeyesdad'], axis=1)
X_test = data_processing(test)
t = data_processing(train).NObeyesdad
t = train_processing(t)

In [8]:
from optuna.integration import lightgbm as lgb
from sklearn.model_selection import KFold

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [10]:
oof_li = []
for fold, (train_index, valid_index) in enumerate(kf.split(X,t)):
    X_train = X.iloc[train_index]
    X_valid = X.iloc[valid_index]
    t_train = t[train_index]
    t_valid = t[valid_index]
    
    lgb_train = lgb.Dataset(X_train, t_train)
    lgb_valid = lgb.Dataset(X_valid, t_valid)
    
    lgb_param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        'verbosity':-1,
        "boosting_type": "gbdt",
        'random_state':0,
        "num_class": 7,
        'device': 'gpu',
    }
    
    gbm = lgb.train(params=lgb_param,
                   train_set=lgb_train,
                   valid_sets=lgb_valid,
                   callbacks=[lgb.early_stopping(stopping_rounds=20,verbose=True), # early_stopping用コールバック関数
#                            lgb.log_evaluation(verbose_eval)
                             ], # コマンドライン出力用コールバック関数
#                    verbose_eval=-1,
                   )
    oof_li.append(gbm.predict(X_test))
    print(f'finish{fold+1}/5')
    if fold == 1:
        break
print(gbm)

[I 2024-02-28 11:36:44,504] A new study created in memory with name: no-name-3a4cb26a-b3f2-45de-bc34-1c13e81a97ce
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 20 rounds


feature_fraction, val_score: 0.271343:  14%|#4        | 1/7 [00:07<00:47,  7.97s/it]

Early stopping, best iteration is:
[65]	valid_0's multi_logloss: 0.271343
Training until validation scores don't improve for 20 rounds


feature_fraction, val_score: 0.271343:  29%|##8       | 2/7 [00:10<00:24,  4.83s/it]

...

Early stopping, best iteration is:
[63]	valid_0's multi_logloss: 0.264105
Training until validation scores don't improve for 20 rounds


min_child_samples, val_score: 0.263135:  80%|########  | 4/5 [00:18<00:04,  4.75s/it]

Early stopping, best iteration is:
[57]	valid_0's multi_logloss: 0.266022
Training until validation scores don't improve for 20 rounds


min_child_samples, val_score: 0.263135: 100%|##########| 5/5 [00:25<00:00,  5.01s/it]

Early stopping, best iteration is:
[59]	valid_0's multi_logloss: 0.264355
finish2/5


In [11]:
X_test.shape

(13840, 17)

In [23]:
oof_array = np.array(oof_li)
mean_oof = np.mean(oof_array, axis=0)
oof_array.shape, mean_oof.shape

((2, 13840, 7), (13840, 7))

In [20]:
np.argmax(mean_oof,axis=1)

(13840,)

In [14]:
for i in range(10):
    print(sum(oof_array[0][i]))

1.0
1.0000000000000002
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999996
1.0
1.0000000000000002
0.9999999999999998
0.9999999999999999


In [15]:
gbm.predict(X_valid)[0]

array([1.25110497e-05, 1.33472849e-05, 2.33165168e-05, 1.58281140e-05,
       1.17624452e-04, 3.14271454e-05, 9.99785945e-01])

In [16]:
np.argmax(gbm.predict(X_valid)[4])

2

In [17]:
t_valid.values[0]

6

In [18]:
predict = gbm.predict(X_valid)
correct = 0
for i in range(len(t_valid.values)):
    if np.argmax(predict[i]) == t_valid.values[i]:
        correct += 1
print(correct/(len(t_valid.values)+1))

0.9109077775102336
